In [1]:
import numpy as np

import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords,twitter_samples
from nltk.tokenize import  TweetTokenizer
from nltk.stem import PorterStemmer
from string import punctuation
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def preprocess_tweete(corbus,tokenizer,steeming,stop_words):
    clean_corbus=[]
    for index,tweet in enumerate (corbus):
        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '',tweet )

        # remove hyperlinks
        tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)

        # remove hashtags
        # only removing the hash # sign from the word
        tweet = re.sub(r'#', '', tweet)
        
        tokenzie_tweet=tokenizer(tweet)
        
        clean_tweet=[]
        for token in tokenzie_tweet:
            if token not in stop_words and token not in punctuation:
                clean_tweet.append(steeming(token))

        
        
        clean_corbus.append(clean_tweet)
    return clean_corbus

        

In [3]:
def freq_dict(clean_tweet,labels):
    freq_dict={}
    for label,tweet in zip(labels,clean_tweet):
        for word in tweet:
            freq_dict[(word,label)]=freq_dict.get((word,label),0)+1
    return freq_dict

In [4]:
def Naive_Bayes(x_train,y_train,freq):
    vocap={word for word,_ in freq.keys()}
    vocap_size=len(vocap)
    number_positove=sum(y_train==1)
    number_negative=sum(y_train==0)
    log_prior=np.log(number_positove/number_negative)
    positive_words=negative_words=0
    for word in vocap:
        if freq.get((word,1),0)!=0:
            positive_words+=1
        if freq.get((word,0),0)!=0:
            negative_words+=1
    likehood={}
    for i in vocap:
        word_to_positve=(freq.get((i,1),0)+1)/(positive_words+vocap_size)
        word_to_negative=(freq.get((i,0),0)+1)/(negative_words+vocap_size)
        log_probabilty=np.log(word_to_positve/word_to_negative)
        likehood[i]=(word_to_positve,word_to_negative,log_probabilty)
    return likehood,log_prior
    
        

In [5]:
positive_tweet=twitter_samples.strings('positive_tweets.json')
negative_tweet=twitter_samples.strings('negative_tweets.json')
all_tweet=np.append(positive_tweet,negative_tweet)
label=np.append(np.ones([len(positive_tweet),1]),np.zeros([len(negative_tweet),1]))

In [6]:
stop_words=stopwords.words('english')
tokenizer=TweetTokenizer(preserve_case=False,
    reduce_len=True,
    strip_handles=True)
steeming=PorterStemmer()

In [7]:
clean_tweet=preprocess_tweete(all_tweet,tokenizer.tokenize,steeming.stem,stop_words)

In [8]:
freq=freq_dict(clean_tweet,label)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(clean_tweet,label,test_size=.2)

In [10]:
likehood,log_perior=Naive_Bayes(x_train,y_train,freq)

In [11]:
def predict(sample,likehood,log_perior):
    values=0
    for token in (sample):
        values+=likehood[token][2]
        
        
    predict=1 if values*log_perior>0 else 0
    return predict
        
    
   

In [12]:
def accuracy(x_test,y_test,likehood,log_perior):
    Y=np.zeros_like(y_test)
    
    for index,sample in enumerate(x_test):
        value=predict(sample,likehood,log_perior)
        Y[index]=value
    accuracy=sum(y_test==Y)/len(y_test)
    return accuracy
        
        
        
        
    

In [13]:
accuracy(x_test,y_test,likehood,log_perior)

0.9985

In [14]:
def Predict_your_tweet(tweet,likehood,log_perior):
    tweet=[tweet]
    tweet=preprocess_tweete(tweet,tokenizer.tokenize,steeming.stem,stop_words)
    y=predict(tweet[0],likehood,log_perior)
    y='Positive'if y>0 else 'Negative'
    return y
    
    
    

In [15]:
Predict_your_tweet('i love you',likehood,log_perior)

'Positive'